In [1]:
!pip install beautifulsoup4

In [2]:
import requests
from bs4 import BeautifulSoup
import re
import pprint
import pandas as pd

**Choose Artists**

---



In [3]:
# Destiny's child link
url_DC='https://www.lyrics.com/album/2571820/Playlist%3A-The-Very-Best-of-Destiny%27s-Child'

In [ ]:
# Radiohead link
url_Radiohead='https://www.lyrics.com/artist/Radiohead/41092'

In [4]:
# send request and get response
response=requests.get(url_DC)
# save html as text 
lyrics_DC_html=response.text
type(lyrics_DC_html)

str

In [5]:
lyrics_DC_html

'\n<!doctype html>\n<!--[if lt IE 7]> <html class="no-js lt-ie9 lt-ie8 lt-ie7" lang="en"> <![endif]-->\n<!--[if IE 7]>    <html class="no-js lt-ie9 lt-ie8" lang="en"> <![endif]-->\n<!--[if IE 8]>    <html class="no-js lt-ie9" lang="en"> <![endif]-->\n<!--[if gt IE 8]><!--> <html class="no-js" lang="en"> <!--<![endif]--><head>\n<meta charset="utf-8">\n<meta http-equiv="X-UA-Compatible" content="IE=edge,chrome=1">\n<title>Various Artists - Playlist: The Very Best of Destiny\'s Child Album | Lyrics.com</title>\n<meta name="description" content="Playlist: The Very Best of Destiny\'s Child is the third compilation album by American R&amp;B girl group Destiny\'s Child. It was released on October 9, 2012 through Columbia Records matching with the fifteenth anniversary of Destiny\'s Child\'s formation. The compilation contained fourteen songs from the group\'s repertoire consisting of four studio albums. Upon its release, the received positive reviews from critics who praised its track list fe

In [6]:
# convert html to a soup object
lyrics_DC_soup = BeautifulSoup(lyrics_DC_html, 'html.parser')
type(lyrics_DC_soup)

bs4.BeautifulSoup

In [7]:
print(lyrics_DC_soup)


<!DOCTYPE doctype html>

<!--[if lt IE 7]> <html class="no-js lt-ie9 lt-ie8 lt-ie7" lang="en"> <![endif]-->
<!--[if IE 7]>    <html class="no-js lt-ie9 lt-ie8" lang="en"> <![endif]-->
<!--[if IE 8]>    <html class="no-js lt-ie9" lang="en"> <![endif]-->
<!--[if gt IE 8]><!--> <html class="no-js" lang="en"> <!--<![endif]--><head>
<meta charset="utf-8"/>
<meta content="IE=edge,chrome=1" http-equiv="X-UA-Compatible"/>
<title>Various Artists - Playlist: The Very Best of Destiny's Child Album | Lyrics.com</title>
<meta content="Playlist: The Very Best of Destiny's Child is the third compilation album by American R&amp;B girl group Destiny's Child. It was released on October 9, 2012 through Columbia Records matching with the fifteenth anniversary of Destiny's Child's formation. The compilation contained fourteen songs from the group's repertoire consisting of four studio albums. Upon its release, the received positive reviews from critics who praised its track list featuring the band's most 

In [8]:
lyrics_DC_soup.head.title.text

"Various Artists - Playlist: The Very Best of Destiny's Child Album | Lyrics.com"

**Get the hyperlinks to lyrics pages**

---



In [9]:
# find all url links using BeautifulSoup
urls=[]
for tag in lyrics_DC_soup.findAll('a', href=True):
    urls.append(tag['href'])
    
#url

In [10]:
# get all the url links to songs
prefix_url ='https://www.lyrics.com'
complete_url_DC_list = []

for hyper_link in urls:
    #print(type(str(hyper_link)))
    
    hyper_list_DC = re.findall(pattern='/lyric/.*', string=hyper_link)
    if  not not hyper_list_DC: # if the list is not empty
        complete_url = prefix_url + hyper_list_DC[0]
        complete_url_DC_list.append(complete_url)

complete_url_DC_list

['https://www.lyrics.com/lyric/27756259/Bootylicious',
 'https://www.lyrics.com/lyric/27756258/Bug+a+Boo+%5BH-town+Screwed+Mix%5D',
 'https://www.lyrics.com/lyric/27756257/Emotion+%5BThe+Neptunes+Remix%5D',
 'https://www.lyrics.com/lyric/27756256/Jumpin%27%2C+Jumpin%27',
 'https://www.lyrics.com/lyric/32463822/Jumpin%27%2C+Jumpin%27+%5BSo+So+Def+Remix%5D',
 'https://www.lyrics.com/lyric/27756255/Independent+Women+Part+I+%5B%231%27s+Edit%5D+%5BEdit%5D',
 'https://www.lyrics.com/lyric/27756254/Say+My+Name+%5B%231%27s+Edit%5D+%5BEdit%5D',
 'https://www.lyrics.com/lyric/27756250/No%2C+No%2C+No%2C+Pt.+2',
 'https://www.lyrics.com/lyric/27756249/Survivor+%5B%231%27s+Edit%5D+%5BEdit%5D',
 'https://www.lyrics.com/lyric/27756248/Lose+My+Breath',
 'https://www.lyrics.com/lyric/27756247/So+Good',
 'https://www.lyrics.com/lyric/27756246/Girl',
 'https://www.lyrics.com/lyric/32463829/Girl+%5BVersion%5D',
 'https://www.lyrics.com/lyric/27756253/Bills%2C+Bills%2C+Bills+%5B%231%27s+Edit%5D+%5BEdit%5D'

**Get the lyrics in all pages**

---



In [11]:
# def to read URL to soup
def read_URL_to_Soup(url):
  response=requests.get(url)
  html = response.text
  html_soup = BeautifulSoup(html, 'html.parser')
  return(html_soup)

# get the lyrics text body
def get_Lyrics(html_soup):
  html_body_lyrics_soup= html_soup.body.find(id="lyric-body-text")
  return(html_body_lyrics_soup)

# get lyrics text and clean it 
def clean_Lyrics(html_body_lyrics_soup):
  html_no_tag = re.sub('<.*?>' ,'', str(html_body_lyrics_soup))
  # remove non alphabetical characters 
  html_clean1=re.sub('\r', '', html_no_tag)
  html_clean2=re.sub('\n', ' ', html_clean1)
  html_clean3=re.sub('\W', ' ',html_clean1)
  return(html_clean3)

# all together get Lyrics
def get_clean_Lyrics(url):
   return clean_Lyrics(get_Lyrics(read_URL_to_Soup(url)))

# get Song title  
def get_song_title(html_soup):
  return(html_soup.head.title.text)


In [12]:
# all lyrics together
song_title=[]
corpus=[]

for url in complete_url_DC_list:
  corpus.append(get_clean_Lyrics(url))
  song_title.append(get_song_title(read_URL_to_Soup(url)))

**Check for repeated songs in titles**

---



In [13]:
song_title

["Destiny's Child - Bootylicious Lyrics | Lyrics.com",
 "Destiny's Child - Bug a Boo [H-town Screwed Mix] Lyrics | Lyrics.com",
 "Destiny's Child - Emotion [The Neptunes Remix] Lyrics | Lyrics.com",
 "Destiny's Child - Jumpin', Jumpin' Lyrics | Lyrics.com",
 "Da Brat - Jumpin', Jumpin' [So So Def Remix] Lyrics | Lyrics.com",
 "Destiny's Child - Independent Women Part I [#1's Edit] [Edit] Lyrics | Lyrics.com",
 "Destiny's Child - Say My Name [#1's Edit] [Edit] Lyrics | Lyrics.com",
 'Wyclef Jean - No, No, No, Pt. 2 Lyrics | Lyrics.com',
 "Destiny's Child - Survivor [#1's Edit] [Edit] Lyrics | Lyrics.com",
 "Destiny's Child - Lose My Breath Lyrics | Lyrics.com",
 "Destiny's Child - So Good Lyrics | Lyrics.com",
 "Destiny's Child - Girl Lyrics | Lyrics.com",
 "Destiny's Child - Girl [Version] Lyrics | Lyrics.com",
 "Destiny's Child - Bills, Bills, Bills [#1's Edit] [Edit] Lyrics | Lyrics.com",
 "Destiny's Child - Soldier [Radio Edit] [Edit] Lyrics | Lyrics.com",
 'Wyclef Jean - Illusion L

**Remove repeated song**

In [14]:
len(corpus)

16

In [15]:
del corpus[11] # the list number of repeated song

In [16]:
corpus

['Kelly  can you handle this  Michelle  can you handle this  Beyoncé  can you handle this  I don t think they can handle this   Barely move  we ve arrived Lookin  sexy  lookin  fly Baddest chick  chick inside DJ  jam tonight Spotted me a tender thang There you are  come on baby Don t you wanna dance with me Can you handle  handle me  You gotta do much better if you gone dance with me tonight You gotta work your jelly if you gone dance with me tonight Read my lips carefully if you like what you see Move  groove  prove you can hang with me By the looks I got you shook up and scared of me Hook up your seat belt  it s time for takeoff  I don t think you ready for this jelly I don t think you ready for this jelly I don t think you ready for this  Cause my body too bootylicious for yo babe  I don t think you ready for this jelly I don t think you ready for this jelly I don t think you ready for this  Cause my body too bootylicious for yo babe  Baby  can you handle this  Baby  can you handle 

**Second Artist**

---

